基于线性假设的线性插值——为了满足10min功率变化考核量

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

data = pd.read_csv("std_data.csv")

df = pd.DataFrame(data)

# 合并日期和时间为单一的datetime列
df['Datetime_'] = pd.to_datetime(df['Datetime'] + ' ' + df['time'])
df.drop(['Datetime', 'time'], axis=1, inplace=True)

# 将Datetime设置为索引
df.set_index('Datetime_', inplace=True)

# 对数据进行升序排序
df.sort_index(ascending=True, inplace=True)

# 初始化一个空的DataFrame来存储插值后的结果
columns = df.columns
interpolated_data = pd.DataFrame(columns=columns)

# 遍历原始数据进行插值
for i in tqdm(range(len(df) - 1), desc='Interpolating'):
    row1 = df.iloc[i]
    row2 = df.iloc[i + 1]
    diff = (row2 - row1) / 3  # 计算差值，并按1/3和2/3的比例进行分配
    
    # 生成两个新的时间点
    time1 = df.index[i] + pd.Timedelta(minutes=5)
    time2 = df.index[i] + pd.Timedelta(minutes=10)
    
    # 计算插值后的数据
    interpolated_row1 = row1 + diff
    interpolated_row2 = row1 + 2 * diff
    
    # 将插值后的数据添加到结果DataFrame中
    interpolated_data = pd.concat([interpolated_data, pd.DataFrame(interpolated_row1).transpose().set_index(pd.Index([time1]))])
    interpolated_data = pd.concat([interpolated_data, pd.DataFrame(interpolated_row2).transpose().set_index(pd.Index([time2]))])

# 调整interpolated_data的index类型
interpolated_data.index = pd.to_datetime(interpolated_data.index)

# 将插值后的数据与原始数据合并，并按时间排序
final_data = pd.concat([df, interpolated_data]).sort_index()

final_data.to_csv("std_data_5min.csv")

Interpolating: 100%|██████████| 35039/35039 [01:50<00:00, 317.58it/s]


In [4]:
import pandas as pd
import numpy as np
PReal = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_realpower"]
Pust = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_ust_predpower"]


# 构造10min考核点
# 由5min构造10min考核点时，由于是后向平均，需要用i点和i点前一个数据求和除以二
# 105119
PReal = np.array(PReal)  # 确保PReal是一个numpy数组
averagesPReal = (PReal[:-1] + PReal[1:]) / 2.0
averagesPReal = np.append(averagesPReal, averagesPReal[-1]) # 求平均会缺一个数据，复制一个最后数据，保持数组长度一致

Pust = np.array(Pust)  # 确保PReal是一个numpy数组
averagesPPust = (Pust[:-1] + Pust[1:]) / 2.0
averagesPPust = np.append(averagesPPust, averagesPPust[-1]) # 保持数组长度一致

averagesPReal,averagesPPust # 10min考核点时间尺度数据，按照i，i+2考核